In [1]:
!nvidia-smi

Tue Dec 10 18:39:33 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    60W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!ps aux | grep python

root          19  4.3  0.7 402904 98672 ?        Sl   18:38   0:01 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         127 23.1  1.1 644096 151712 ?       Ssl  18:39   0:01 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-40f0384f-b999-4f03-8570-605e4c87218d.json
root         149  0.0  0.0  39192  6468 ?        S    18:39   0:00 /bin/bash -c ps aux | grep python


# **Import and Load data**

In [3]:
import pandas as pd
import numpy as np
from PIL import Image
import shutil
from torchvision import models
import torch.nn as nn
import torch
import json
import time
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from zipfile import ZipFile 
from keras.preprocessing.image import load_img, save_img, img_to_array
from matplotlib import pyplot as plt
from google.colab import drive
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()

In [5]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# **Define Models**

In [0]:
with ZipFile("/content/gdrive/My Drive/Painting Dataset/train.zip", 'r') as zip_f: 
  zip_f.extractall(path='/content') 
with ZipFile("/content/gdrive/My Drive/Painting Dataset/val.zip", 'r') as zip_f: 
  zip_f.extractall(path='/content')    

In [0]:
train_directory = 'train'
val_directory = 'val'
preprocess = transforms.Compose([
              transforms.RandomResizedCrop(224), # random resized crop is used 
                                                 # for data augmentation
              transforms.ToTensor(), 
              transforms.Normalize((0.4914, 0.4822, 0.4465),
                                   (0.2023, 0.1994, 0.2010))
              ])
train_data = ImageFolder(train_directory, transform=preprocess)

val_data = ImageFolder(val_directory, transform=preprocess)

In [8]:
n_category = 50
# Resnet 18
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Linear(512, n_category)
# Resnet 50
resnet50 = models.resnet50(pretrained=True)
resnet50.fc = nn.Linear(2048, n_category)
# Resnet 101
resnet101 = models.resnet101(pretrained=True)
resnet101.fc = nn.Linear(2048, n_category)
# GoogleNet
googleNet = models.googlenet(pretrained=True)
googleNet.fc = nn.Linear(1024, n_category)
# VGG 11 with batch normalization
vgg11 = models.vgg11_bn(pretrained=False)
vgg11.classifier[6] = nn.Linear(4096, n_category)
# VGG 19 with batch normalization
vgg19 = models.vgg19_bn(pretrained=True)
vgg19.classifier[6] = nn.Linear(4096, n_category)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 164MB/s]
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 163MB/s]
Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth
100%|██████████| 170M/170M [00:01<00:00, 160MB/s]
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 166MB/s]
Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:07<00:00, 79.6MB/s]


# **Model Training**

In [9]:
!pip install tqdm --upgrade

     |████████████████████████████████| 61kB 3.1MB/s 
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
from tqdm.notebook import tqdm
import torch.optim as optim
from sklearn.metrics import f1_score, recall_score, precision_score
def train(classifier, criterion, optimizer, trainLoader, device):
    epoch_loss = 0
    epoch_correct = 0
    classifier.train()
    record_pred = torch.FloatTensor().cpu()
    record_gt = torch.LongTensor().cpu()
    for i, (x, y) in tqdm(enumerate(trainLoader)):
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        predictions = classifier(x)
        loss = criterion(predictions, y)
        # print('predictions.size()',predictions.size())
        epoch_correct += torch.eq(torch.argmax(predictions, dim=1), y).sum().item()

        record_pred = torch.cat([record_pred, torch.argmax(predictions, dim=1).cpu().type(torch.FloatTensor)], dim=0)
        record_gt = torch.cat([record_gt, y.cpu()], dim=0)

        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
    record_pred = record_pred.view(-1)
    record_gt = record_gt.view(-1)
    f1 = f1_score(record_gt, record_pred, average='weighted')
    precision = precision_score(record_gt, record_pred, average='weighted')
    recall = recall_score(record_gt, record_pred, average='weighted')
    print('Train f1', f1)
    print('Train precision', precision)
    print('Train recall', recall)
    return epoch_loss / len(trainLoader.dataset), epoch_correct / len(trainLoader.dataset), f1, precision, recall

def evaluate(classifier, criterion, optimizer, validLoader, device):
  epoch_loss = 0
  epoch_correct = 0
  classifier.eval()
  record_pred = torch.FloatTensor().cpu()
  record_gt = torch.LongTensor().cpu()
  with torch.no_grad():
      for i, (x, y) in tqdm(enumerate(validLoader)):
          x, y = x.to(device), y.to(device)

          predictions = classifier(x)
          
          loss = criterion(predictions, y)
          record_pred = torch.cat([record_pred, torch.argmax(predictions, dim=1).cpu().type(torch.FloatTensor)], dim=0)
          record_gt = torch.cat([record_gt, y.cpu()], dim=0)

          epoch_loss += loss.item()
          epoch_correct += torch.eq(torch.argmax(predictions, dim=1), y).sum().item()
      record_pred = record_pred.view(-1)
      record_gt = record_gt.view(-1)
      f1 = f1_score(record_gt, record_pred, average='weighted')
      precision = precision_score(record_gt, record_pred, average='weighted')
      recall = recall_score(record_gt, record_pred, average='weighted')
      print('Val f1', f1)
      print('Val precision', precision)
      print('Val recall', recall)
  return epoch_loss / len(validLoader.dataset), epoch_correct / len(validLoader.dataset)

  # Plot funciton
def plot_result(train_acc, val_acc, train_loss, val_loss):
  epochs = np.arange(0, 60)
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6), dpi=600)
  ax1.set_title('Train and Valid Accuracies over Epochs')
  ax1.set_xlabel('Epochs')
  ax1.set_ylabel('Accuracy')
  ax1.plot(epochs, train_acc, label='training accuracy')
  ax1.plot(epochs, val_acc, label='validation accuracy')
  ax1.legend()

  ax2.set_title('Train and Valid Loss over Epochs')
  ax2.set_xlabel('Epochs')
  ax2.set_ylabel('Loss')
  ax2.plot(epochs, train_loss, label='training loss')
  ax2.plot(epochs, val_loss, label='validation loss')
  ax2.legend()
  fig.savefig('googleNet_loss_acc.png', dpi=600)

In [0]:
def train_model(classifier, criterion, optimizer, 
                trainSet, testSet, n_epochs = 60):
    params = {  
        'batch_size':64,
        'shuffle':True,
        'num_workers':0,
    }
    train_ls_acc, valid_ls_acc = [], []
    # all_train_acc, all_val_acc = [], []
    # all_train_loss, all_val_loss = [], []
    for epoch in range(n_epochs):
        train_loader = DataLoader(trainSet, **params)
        val_loader = DataLoader(testSet, **params)

        start_time = time.time()
        
        train_loss, train_acc, f1_t, p_t, r_t = train(classifier, criterion, optimizer, train_loader, device)
        valid_loss, valid_acc = evaluate(classifier, criterion, optimizer, val_loader, device)
        
        end_time = time.time()
        elapsed_time = end_time - start_time
        epoch_mins = int(elapsed_time/60)
        epoch_secs = int(elapsed_time - epoch_mins*60)

        train_ls_acc.append((train_loss, train_acc))
        valid_ls_acc.append((valid_loss, valid_acc))
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    return train_ls_acc, valid_ls_acc

**Train VGG11**

In [12]:
model = resnet50.to(device)
criterion = nn.CrossEntropyLoss().to(device)
lr = 0.001
optimizer = optim.SGD(model.parameters(), lr)
train_ls_acc, valid_ls_acc = train_model(model, criterion, optimizer, train_data, val_data)


Train f1 0.04622727994105351
Train precision 0.044926760301126356
Train recall 0.12951450562462996


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Val f1 0.0570318773572302
Val precision 0.06361265033217268
Val recall 0.1636094674556213
Epoch: 01 | Epoch Time: 10m 35s
	Train Loss: 0.057 | Train Acc: 12.95%
	 Val. Loss: 0.054 |  Val. Acc: 16.36%



Train f1 0.09236851073168544
Train precision 0.18230403804787843
Train recall 0.1913854351687389



Val f1 0.11534418758154029
Val precision 0.21053264906260286
Val recall 0.2195266272189349
Epoch: 02 | Epoch Time: 10m 0s
	Train Loss: 0.052 | Train Acc: 19.14%
	 Val. Loss: 0.050 |  Val. Acc: 21.95%


...


Train f1 0.8227322916385538
Train precision 0.8265018622102863
Train recall 0.8283747779751333



Val f1 0.7446974734537748
Val precision 0.7486291283096637
Val recall 0.7526627218934911
Epoch: 49 | Epoch Time: 9m 39s
	Train Loss: 0.010 | Train Acc: 82.84%
	 Val. Loss: 0.014 |  Val. Acc: 75.27%



Train f1 0.8274492068143906
Train precision 0.8320171092669062
Train recall 0.8331113084665482



Val f1 0.7398060125283119
Val precision 0.7429469874856103
Val recall 0.7473372781065089
Epoch: 50 | Epoch Time: 9m 40s
	Train Loss: 0.010 | Train Acc: 83.31%
	 Val. Loss: 0.014 |  Val. Acc: 74.73%


KeyboardInterrupt: ignored

In [0]:
all_train_acc = [l[1] for l in train_ls_acc]
all_train_loss = [l[0] for l in train_ls_acc]
all_val_acc = [l[1] for l in valid_ls_acc]
all_val_loss = [l[0] for l in valid_ls_acc]
plot_result(all_train_acc,all_val_acc,all_train_loss,all_val_loss)